# Feature importance 평가를 위한 SHAP
- kfold validation 실행 후 SHAP 
- 연산에 소요되는 시간 확인을 위해 우선적으로 catboost model에 대해서만 실행

In [ ]:
혹시 연구실 컴퓨터에 shap 설치 안되어 있으면 설치해서 실행하길...

In [1]:
import pandas as pd
import shap
import numpy as np
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from deepforest import CascadeForestRegressor
from sklearn.model_selection import KFold
np.int = int
np.bool = bool

c:\Users\SAL\AppData\Local\Programs\Python\Python38\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Users\SAL\AppData\Local\Programs\Python\Python38\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def

In [2]:
df = pd.read_csv("dataset.csv")
X = df[['cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age']]
y = df['CCS']

In [3]:
# 5-fold cross-validation 실행
kf = KFold(n_splits=5, shuffle=True, random_state=42)
shap_values_all = []
feature_names = X.columns

In [4]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
shap_values_all = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"Fold {fold+1}...")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # 1) DeepForest 모델 학습
    model = CascadeForestRegressor(random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)

    # 2) 함수형 SHAP Explainer 생성
    #    DeepForest는 TreeExplainer를 직접 지원하지 않으므로 shap.Explainer가
    #    모델 구조를 보고 KernelExplainer 등으로 자동 선택합니다.
    explainer = shap.Explainer(model.predict, X_train, feature_names=X.columns.tolist())

    # 3) SHAP 값 계산 (Explanation 객체)
    explanation = explainer(X_test)

    # 4) 절댓값 취해 리스트에 저장
    shap_values_all.append(np.abs(explanation.values))

# 5) 모든 fold 결과를 합치고 feature별 평균 SHAP 값 계산
all_shap = np.vstack(shap_values_all)  # (총 샘플 수, 피처 수)
mean_shap_per_feature = all_shap.mean(axis=0)

# 6) 결과 출력
for name, val in zip(X.columns, mean_shap_per_feature):
    print(f"{name:20s}: {val:.4f}")

# 6) 결과 출력
for name, val in zip(X.columns, mean_shap_per_feature):
    print(f"{name:20s}: {val:.4f}")

Fold 1...
[2025-07-30 03:37:17.853] Start to fit the model:
[2025-07-30 03:37:17.853] Fitting cascade layer = 0 
[2025-07-30 03:37:18.521] layer = 0  | Val MSE = 28.01044 | Elapsed = 0.666 s
[2025-07-30 03:37:18.521] Fitting cascade layer = 1 
[2025-07-30 03:37:19.204] layer = 1  | Val MSE = 21.74305 | Elapsed = 0.684 s
[2025-07-30 03:37:19.204] Fitting cascade layer = 2 
[2025-07-30 03:37:19.804] layer = 2  | Val MSE = 24.03155 | Elapsed = 0.600 s
[2025-07-30 03:37:19.804] Early stopping counter: 1 out of 2
[2025-07-30 03:37:19.804] Fitting cascade layer = 3 
[2025-07-30 03:37:20.487] layer = 3  | Val MSE = 26.69418 | Elapsed = 0.683 s
[2025-07-30 03:37:20.487] Early stopping counter: 2 out of 2
[2025-07-30 03:37:20.487] Handling early stopping
[2025-07-30 03:37:20.488] The optimal number of layers: 2
[2025-07-30 03:37:23.355] Start to evalute the model:
[2025-07-30 03:37:23.361] Evaluating cascade layer = 0 
[2025-07-30 03:37:23.567] Evaluating cascade layer = 1 
[2025-07-30 03:37:25

Exact explainer:   4%|▍         | 9/206 [00:00<?, ?it/s]

[2025-07-30 03:37:30.789] Start to evalute the model:
[2025-07-30 03:37:30.789] Evaluating cascade layer = 0 
[2025-07-30 03:37:31.016] Evaluating cascade layer = 1 
[2025-07-30 03:37:31.219] Start to evalute the model:
[2025-07-30 03:37:31.219] Evaluating cascade layer = 0 
[2025-07-30 03:37:31.335] Evaluating cascade layer = 1 


Exact explainer:   5%|▌         | 11/206 [00:10<01:05,  2.98it/s]

[2025-07-30 03:37:31.460] Start to evalute the model:
[2025-07-30 03:37:31.471] Evaluating cascade layer = 0 
[2025-07-30 03:37:31.682] Evaluating cascade layer = 1 
[2025-07-30 03:37:31.939] Start to evalute the model:
[2025-07-30 03:37:31.940] Evaluating cascade layer = 0 
[2025-07-30 03:37:32.038] Evaluating cascade layer = 1 


...

[2025-07-30 03:49:14.137] Evaluating cascade layer = 1 
[2025-07-30 03:49:14.237] Start to evalute the model:
[2025-07-30 03:49:14.237] Evaluating cascade layer = 0 
[2025-07-30 03:49:14.486] Evaluating cascade layer = 1 
[2025-07-30 03:49:14.687] Start to evalute the model:
[2025-07-30 03:49:14.687] Evaluating cascade layer = 0 
[2025-07-30 03:49:14.802] Evaluating cascade layer = 1 


Exact explainer: 100%|██████████| 206/206 [02:19<00:00,  1.48it/s]

[2025-07-30 03:49:14.937] Start to evalute the model:
[2025-07-30 03:49:14.937] Evaluating cascade layer = 0 
[2025-07-30 03:49:15.154] Evaluating cascade layer = 1 
[2025-07-30 03:49:15.354] Start to evalute the model:
[2025-07-30 03:49:15.354] Evaluating cascade layer = 0 
[2025-07-30 03:49:15.470] Evaluating cascade layer = 1 


Exact explainer: 207it [02:20,  1.36it/s]                         

cement              : 5.8663
blast furnace slag  : 2.3694
fly ash             : 0.8868
water               : 4.2120
superplasticizer    : 2.2264
coarse aggregate    : 0.9953
fine aggregate      : 1.6464
age                 : 7.6725
cement              : 5.8663
blast furnace slag  : 2.3694
fly ash             : 0.8868
water               : 4.2120
superplasticizer    : 2.2264
coarse aggregate    : 0.9953
fine aggregate      : 1.6464
age                 : 7.6725


In [5]:
# SHAP 평균값 계산 -> 최종 importance 계산
mean_shap = np.mean(np.array(shap_values_all), axis=0)  # shape: (n_samples, n_features)
feature_importance = pd.Series(mean_shap.mean(axis=0), index=feature_names).sort_values(ascending=False)


In [6]:
print("\n Feature Importance (Mean SHAP Values across 5 folds):")
print(feature_importance)


 Feature Importance (Mean SHAP Values across 5 folds):
age                   7.672503
cement                5.866272
water                 4.212001
blast furnace slag    2.369424
superplasticizer      2.226372
fine aggregate        1.646417
coarse aggregate      0.995342
fly ash               0.886771
dtype: float64
